In [1]:
import requests,re,time,numpy as np

In [2]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Referer':'https://www.costcotravel.com',
    'Host':'www.costcotravel.com',
}

In [3]:
r3 = requests.get('https://www.costcotravel.com/h=4005',headers=headers)
cks = r3.cookies
print(cks.keys())

['BIGipServerpool-prod-app', 'JSESSIONID', 'SESSION_TIME_OUT_DETECTED']


In [4]:
r3.text

' <!doctype html> <html xmlns="http://www.w3.org/1999/xhtml" style="height: 100%;"><head><meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="telephone=no" name="format-detection"/><meta content="Costco Travel offers everyday savings on top-quality, brand-name vacations, cruises and rental cars, exclusively for Costco members." name="description"/><meta content="#0060a9" name="theme-color"/><meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/><meta content="no-cache" http-equiv="Pragma"/><meta content="0" http-equiv="Expires"/><meta content="www.costcotravel.com: Vacation Packages,Cruises,Rental Cars" name="keywords"/><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><meta content="GENERAL" name="rating"/><meta content="" name="robots"/><link \n\t\trel="alternate"\t \n\t\n\t\tmedia="only screen and (min-device-width: 641px)"\t \n\t\n\t\tid=

In [5]:
res = re.search('\(\"Csrf-token\", \"(\S{128})\"\)',r3.text)
csrf = res.group(1)
print(csrf)
ts = re.search('namespace.sessionTimestamp = (\d{10,20});',r3.text).group(1)
print(ts)

f83f366665da11f7c9aa7c446e50fd7e9eb2e067f5f2b223e6853d7c11ff9f52e588df812cb11e3542a093043da3aafbd9e055d85a02b6f4667c54460f0866a0
1504827020656


In [6]:
time.sleep(1)

In [7]:
headers.update({'X-Csrf-Token':csrf})
headers.update({'Referer':'https://www.costcotravel.com/h=4005'})

In [8]:
data = { 
    'rcs':'1',
    'driverAge':'25',
    'pickupZip':'60615',
    'pickupCityLocationTypeSearch':'2',
    'dropoffZip':'60615',
    'dropoffCityLocationTypeSearch':'2',
    'pickupCountry':'US',
    'dropoffCountry':'US',
    'pickupCityRadius':'25',
    'dropoffCityRadius':'25',
    'pickupAsAirport':'false',
    'dropoffAsAirport':'false',
    'pickupDate':'09/11/2017',
    'dropoffDate':'10/09/2017',
    'pickupTime':'09:00 AM',
    'dropoffTime':'09:00 AM',
    'pickupLocationCode':'60615',
    'pickupLocationName':'60615 (CHICAGO, IL, US)',
    'pickupLocationType':'zipCode',
    'pickupLocationCityCode':'CHICAGO',
    'pickupLocationStateCode':'',
    'pickupLatitude':'',
    'pickupLongitude':'',
    'dropoffLocationCode':'60615',
    'dropoffLocationName':'60615 (CHICAGO, IL, US)',
    'dropoffLocationType':'zipCode',
    'dropoffLocationCityCode':'CHICAGO',
    'dropoffLocationStateCode':'',
    'dropoffLatitude':'',
    'dropoffLongitude':'',
    'fromHomePage':'true',
    'fromCarVendorMainMenu':'true',
    'carSearchInModifyFlow':'false',
    'suppressOutput':'false',
    #'uid':'1504750265762_835.0398291228158'
}

In [9]:
cks.set('Csrf-token',csrf)
if 'SESSION_TIME_OUT_DETECTED' in cks: cks.pop('SESSION_TIME_OUT_DETECTED')
cks.set('SESSION_TIMESTAMP',str(ts))
print(cks.keys())

['Csrf-token', 'SESSION_TIMESTAMP', 'BIGipServerpool-prod-app', 'JSESSIONID']


In [10]:
#r = requests.post('https://www.costcotravel.com/carSearch.act', data = data)
r = requests.post('https://www.costcotravel.com/rentalCarSearch.act', data = data, headers = headers, cookies = cks)

In [11]:
r.text

'<div><div id="search_widget" class="search-widget"><div data-default-tab="vacation-packages" class="deselectable-mobile" data-hook="tab-switcher"><div class="search-nav"><p class="hide-desktop text-center">Start Searching</p><ul><li data-tab="vacation-packages" class="search-vacation-packages"><a href="#search_widget">Vacation Packages</a></li><li data-tab="cruises" class="search-cruises"><a href="#search_widget">Cruises</a></li><li data-tab="rental-cars" class="search-rental-cars"><a href="#search_widget">Rental Cars</a></li><li data-tab="hotels" class="search-hotels"><a href="#search_widget">Hotels</a></li></ul></div><div class="search-body"><div data-pane="rental-cars" class="pane bound pane-search-rental-cars" id="search_rental_cars_view"><form id="search_rental_cars_form" method="post"><div class="row thirds thirds-lg gutter-line"><div class="col col-3 col-lg col-lg-1 top-spacing"><div data-hook="car_geolocation_wrapper" class="form-required rc-geo-wrapper geolocation-wrapper" id

In [12]:
# res = re.search('{"id":"S(\d{1})","type":"city","agency":"(\S{1,20})","agencyCode":"(\S{1,6})","title":"(\S{4,30})","isOpen":true,"unAvailableMessage":"","vendorCode":"(\S{2})","address"}',r.text)

In [13]:
# res = re.search('{\"id\":\"S(\d{1})\",\"type\":\"city\",\"agency\":\"(\S{1,20})\",\"agencyCode\":\"(\S{1,6})\",\
# \"title\":\"(\S{4,30})\",\"isOpen\":true,\"unAvailableMessage\":\"\",\"vendorCode\":\"(\S{2})\",\"address\"',r.text)
# print(res.groups())

In [14]:
res = re.findall('{\"id\":\"S(\d{1,3})\",\"type\":\"(city|airport)\",\"agency\":\"(\S{1,40})\",\"agencyCode\":\"(\S{1,8})\",\
\"title\":\"(.{4,50})\",\"isOpen\":true,\"unAvailableMessage\":\"\",\"vendorCode\":\"(\S{2,3})\",\"address\"',r.text)

In [15]:
results = dict()
for tp in res[18:50]:
    results[int(tp[0])] = {'type':tp[1],'brand':tp[2],'code':tp[3],'name':tp[4],'bcode':tp[5]}
#print(len(results),results.keys())
#print(results)
print(len(results))

32


In [16]:
r.cookies

<RequestsCookieJar[]>

In [17]:
headers2 = {
    'Referer':'https://www.costcotravel.com/h=3001',
    'Host':'www.costcotravel.com',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'X-Csrf-Token':csrf
}
headers.update({'Referer':'https://www.costcotravel.com/h=3001'})

In [18]:
for k,v in results.items():
    #print(k,v)
    print('[{{\"vendorId\":\"{}\",\"agencyCodes\":[\"{}\"]}}]'.format(v['bcode'],v['code']))
    data2 = {
        'cas':'Load_Forword_Navigation_From_Agency_Results',
        #'carAgenciesForVendors':'[{"vendorId":"ET","agencyCodes":["E11576"]}]',
        'carAgenciesForVendors':'[{{\"vendorId\":\"{}\",\"agencyCodes\":[\"{}\"]}}]'.format(v['bcode'],v['code']),
        #'uid':'1504759332852_756.0475947513028'
    }
    r2 = requests.post('https://www.costcotravel.com/rentalCarAgencySelection.act',data=data2,cookies=cks,headers=headers)
    print(k,r2)
    #print(r2.text)
    prices = re.findall('<h3>(.{5,40})</h3></div></div></div><div style=\"height: 94px;\" class=\"col col-lg col-1 col-lg-1 text-center height-item test\"><a data-responsive-referrer=\"carMatrix\" data-category-id=\"\S{5,50}\" data-selected=\"false\" data-product-id=\"\S{5,50}\" data-price=\"(\S{4,30})\"',r2.text)
    #print(prices)
    if len(prices) == 0:
        print("NO DATA!!!")
        print(r2.text)
    else:
        print("{} OK".format(k))
        for pr in prices:
            try:
                v[pr[0]] = float(pr[1])
            except:
                v[pr[0]] = pr[1]
    print(v)
    time.sleep(3)

[{"vendorId":"AL","agencyCodes":["CHIC72"]}]
19 <Response [200]>
19 OK
{'type': 'city', 'brand': 'alamo', 'code': 'CHIC72', 'name': 'Chicago N Lasalle', 'bcode': 'AL', 'Economy Car': 702.81, 'Compact Car': 702.84, 'Standard Car': 993.73, 'Fullsize Car': 1106.26, 'Intermediate SUV': 1049.99, 'Standard SUV': 1331.32, 'Mini Van': 1059.57, 'Fullsize SUV': 1858.52, 'Standard Convertible': 2916.31}
[{"vendorId":"ET","agencyCodes":["E115CA"]}]
20 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p st

[{"vendorId":"ET","agencyCodes":["E115C6"]}]
21 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

[{"vendorId":"ET","agencyCodes":["E115CU"]}]
22 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

[{"vendorId":"ET","agencyCodes":["E11557"]}]
23 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

[{"vendorId":"ET","agencyCodes":["E115C7"]}]
24 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

...

[{"vendorId":"AV","agencyCodes":["CHIC07"]}]
46 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

[{"vendorId":"AV","agencyCodes":["CHIS04"]}]
47 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

[{"vendorId":"BG","agencyCodes":["CHIS02"]}]
48 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

[{"vendorId":"ET","agencyCodes":["E115C9"]}]
49 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

[{"vendorId":"AV","agencyCodes":["CHIW12"]}]
50 <Response [200]>
NO DATA!!!
<template-search/><div data-hook="page_content" data-id="page_content" id="page_content"><div class="rental-cars bound"></div><div id="car_rental_grid"><div class="section-description hide-mobile"><div class="bound"><div class="section-title"><a data-hook="go_back" class="arrow-left">Back</a><h2>Low Price Finder<sup class="lowPriceTm">TM</sup></h2></div><div class="section-chat hide-mobile"><img class="chat-image" style="display:none;" alt="Agent Image" hideIfMissing="true" id="needHelpAgentImage"/><h2>Need Help?</h2><p style="display:none;" id="agentExpertsInfo">Our experienced travel experts are here for you.</p><p id="needHelpBannerCallTollFree"><b>Call  1-877-849-2730</b><br/>
		 </p><p style="font-size: 11px" id="needHelpBannerWeekdays">Weekdays
		        05:00 AM - 09:00 PM
		    </p><p style="font-size: 11px" id="needHelpBannerWeekends">Weekends 06:00 AM - 05:30 PM<br/>Pacific Time</p><p style="display:n

In [19]:
for k,v in results.items():
    print(v['name'])

Chicago N Lasalle
Lake And Franklin
Gg Archer Heights
Gg Western And Roosevelt
Chicago Magnificent Mile
Gold Coast
Chicago Midway Intl Airport
Midway Airport
Dd Gold Coast Auto Body
Midway Intl Arpt
Midway Intl Arpt
Dd Old Town
Grossinger City Autoplex
Chicago Grossinger Autoplex
Oak Lawn
Oak Lawn
Euro Collission
Gg Oak Lawn
Sat Howard Orloff
Dd Bucktown
Gg Berwyn
Gg Blue Island
Nn Fox Ford
Nn Lincoln Park
Logan Square
Logan Square Chicago Il
Chicago/lakeview
Lakeview/chicago
Sears Auto-westfield Chicago Ridge Mall
Chicago Ridge Il Sears
Nn Gerber Logan Square
Oak Park


In [20]:
desiredcar = 'Compact Car'
results2 = dict()
for k,v in results.items():
    if desiredcar in v:
        results2[k] = v
order = np.array([x['Compact Car'] for x in results2.values()]).argsort()

In [21]:
print(results2)

{19: {'type': 'city', 'brand': 'alamo', 'code': 'CHIC72', 'name': 'Chicago N Lasalle', 'bcode': 'AL', 'Economy Car': 702.81, 'Compact Car': 702.84, 'Standard Car': 993.73, 'Fullsize Car': 1106.26, 'Intermediate SUV': 1049.99, 'Standard SUV': 1331.32, 'Mini Van': 1059.57, 'Fullsize SUV': 1858.52, 'Standard Convertible': 2916.31}}


In [22]:
brands = np.array([x['brand'] for x in results2.values()])[order]
nums = np.array([x for x in results2.keys()])[order]
titles = np.array([x['name'] for x in results2.values()])[order]
prices = np.array([x['Compact Car'] for x in results2.values()])[order]
print(data['pickupDate'],data['pickupTime'],data['dropoffDate'],data['dropoffTime'])
for k in zip(nums,prices,brands,titles):
    print(k)

09/11/2017 09:00 AM 10/09/2017 09:00 AM
(19, 702.84000000000003, 'alamo', 'Chicago N Lasalle')


09/11/2017 09:00 AM 10/09/2017 09:00 AM
(15, 659.90999999999997, 'enterprise', 'Union Station/pres Tower')
(14, 659.90999999999997, 'enterprise', 'Dd West Loop A/b(ex15ca)')
(13, 659.90999999999997, 'enterprise', 'Chicago Loop')
(16, 670.80000000000007, 'enterprise', 'Millennium Park')
(4, 678.0, 'budget', 'Chicago')
(8, 678.0, 'budget', 'Chicago')
(3, 688.52999999999997, 'avis', 'Chicago - 18th Street & Wabash Avenue')
(11, 688.52999999999997, 'avis', 'Chicago Union Train Station')
(10, 700.18000000000006, 'enterprise', 'Gg Gresham')
(5, 700.18000000000006, 'enterprise', 'Gg South Western')
(7, 700.18000000000006, 'enterprise', 'Gg Chatham Village Sq')
(9, 700.18000000000006, 'enterprise', 'Gg Stony Island')
(19, 702.84000000000003, 'alamo', 'Chicago N Lasalle')
(2, 704.54999999999995, 'enterprise', 'Gg South Loop')
(1, 704.54999999999995, 'enterprise', 'Gg Hyde Park')
(18, 716.95000000000005, 'budget', 'North Loop Budget')
(17, 716.95000000000005, 'avis', 'North Loop-downtown')
(12, 745.52999999999997, 'budget', 'Chicago Union Train Station')

In [23]:
"""
09/11/2017 09:00 AM 10/09/2017 09:00 AM
(20, 670.80000000000007, 'enterprise', 'Lake And Franklin')
(23, 670.80000000000007, 'enterprise', 'Chicago Magnificent Mile')
(24, 670.80000000000007, 'enterprise', 'Gold Coast')
(21, 700.18000000000006, 'enterprise', 'Gg Archer Heights')
(22, 700.18000000000006, 'enterprise', 'Gg Western And Roosevelt')
(27, 709.97000000000003, 'enterprise', 'Dd Gold Coast Auto Body')
(26, 896.54999999999995, 'budget', 'Midway Airport')
(29, 1041.0899999999999, 'alamo', 'Midway Intl Arpt')
(28, 1148.0, 'enterprise', 'Midway Intl Arpt')
(25, 1166.74, 'avis', 'Chicago Midway Intl Airport')
"""
"""
09/11/2017 09:00 AM 10/09/2017 09:00 AM
(15, 659.90999999999997, 'enterprise', 'Union Station/pres Tower')
(14, 659.90999999999997, 'enterprise', 'Dd West Loop A/b(ex15ca)')
(13, 659.90999999999997, 'enterprise', 'Chicago Loop')
(16, 670.80000000000007, 'enterprise', 'Millennium Park')
(4, 678.0, 'budget', 'Chicago')
(8, 678.0, 'budget', 'Chicago')
(3, 688.52999999999997, 'avis', 'Chicago - 18th Street & Wabash Avenue')
(11, 688.52999999999997, 'avis', 'Chicago Union Train Station')
(10, 700.18000000000006, 'enterprise', 'Gg Gresham')
(5, 700.18000000000006, 'enterprise', 'Gg South Western')
(7, 700.18000000000006, 'enterprise', 'Gg Chatham Village Sq')
(9, 700.18000000000006, 'enterprise', 'Gg Stony Island')
(19, 702.84000000000003, 'alamo', 'Chicago N Lasalle')
(2, 704.54999999999995, 'enterprise', 'Gg South Loop')
(1, 704.54999999999995, 'enterprise', 'Gg Hyde Park')
(18, 716.95000000000005, 'budget', 'North Loop Budget')
(17, 716.95000000000005, 'avis', 'North Loop-downtown')
(12, 745.52999999999997, 'budget', 'Chicago Union Train Station')
"""

"\n09/11/2017 09:00 AM 10/09/2017 09:00 AM\n(15, 659.90999999999997, 'enterprise', 'Union Station/pres Tower')\n(14, 659.90999999999997, 'enterprise', 'Dd West Loop A/b(ex15ca)')\n(13, 659.90999999999997, 'enterprise', 'Chicago Loop')\n(16, 670.80000000000007, 'enterprise', 'Millennium Park')\n(4, 678.0, 'budget', 'Chicago')\n(8, 678.0, 'budget', 'Chicago')\n(3, 688.52999999999997, 'avis', 'Chicago - 18th Street & Wabash Avenue')\n(11, 688.52999999999997, 'avis', 'Chicago Union Train Station')\n(10, 700.18000000000006, 'enterprise', 'Gg Gresham')\n(5, 700.18000000000006, 'enterprise', 'Gg South Western')\n(7, 700.18000000000006, 'enterprise', 'Gg Chatham Village Sq')\n(9, 700.18000000000006, 'enterprise', 'Gg Stony Island')\n(19, 702.84000000000003, 'alamo', 'Chicago N Lasalle')\n(2, 704.54999999999995, 'enterprise', 'Gg South Loop')\n(1, 704.54999999999995, 'enterprise', 'Gg Hyde Park')\n(18, 716.95000000000005, 'budget', 'North Loop Budget')\n(17, 716.95000000000005, 'avis', 'North L